<a href="https://colab.research.google.com/github/vandanacm/Fall_2025/blob/main/HW3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import sys
import math
import time
import random

def read_graph(filepath):
    """Reads the graph from the given file path."""
    with open(filepath, 'r') as f:
        lines = f.readlines()

    # First line is N
    try:
        N = int(lines[0].strip())
    except ValueError:
        # Fallback if first line is not just a number
        N = 1000

    # Initialize distance matrix
    dist_matrix = [[0.0] * (N + 1) for _ in range(N + 1)]

    # Parse edges
    # Format: Node1 Node2 Distance
    # Skip header if present (Line 2 usually)
    start_idx = 1
    if "Node1" in lines[1]:
        start_idx = 2

    for line in lines[start_idx:]:
        parts = line.strip().split()
        if len(parts) != 3:
            continue
        u, v, d = int(parts[0]), int(parts[1]), float(parts[2])
        dist_matrix[u][v] = d
        dist_matrix[v][u] = d

    return N, dist_matrix

def calculate_cost(path, dist_matrix):
    cost = 0.0
    for i in range(len(path) - 1):
        cost += dist_matrix[path[i]][path[i+1]]
    return cost

def nearest_neighbor(N, dist_matrix, start_node=1):
    unvisited = set(range(1, N + 1))
    unvisited.remove(start_node)
    path = [start_node]
    current = start_node

    while unvisited:
        nearest = None
        min_dist = float('inf')

        # Optimization: For very large N, this is O(N). Total O(N^2).
        # 1000^2 = 1M ops, very fast.
        for neighbor in unvisited:
            d = dist_matrix[current][neighbor]
            if d < min_dist:
                min_dist = d
                nearest = neighbor

        current = nearest
        path.append(current)
        unvisited.remove(current)

    path.append(start_node) # Return to start
    return path

def two_opt(path, dist_matrix, time_limit_seconds=25):
    best_path = path[:]
    best_cost = calculate_cost(best_path, dist_matrix)
    improved = True
    start_time = time.time()

    evaluated_cycles = 0

    while improved:
        improved = False
        if time.time() - start_time > time_limit_seconds:
            break

        N = len(path) - 1 # Number of nodes (excluding return to start)

        for i in range(1, N - 1):
            if time.time() - start_time > time_limit_seconds:
                break
            for j in range(i + 1, N):
                if j - i == 1: continue # Adjacent edges, no change

                evaluated_cycles += 1

                # Current edges: (i-1, i) and (j, j+1)
                # New edges: (i-1, j) and (i, j+1)
                # Path indices: ... i-1, [i ... j], j+1 ...

                u1, v1 = best_path[i-1], best_path[i]
                u2, v2 = best_path[j], best_path[j+1]

                current_delta = dist_matrix[u1][v1] + dist_matrix[u2][v2]
                new_delta = dist_matrix[u1][u2] + dist_matrix[v1][v2]

                if new_delta < current_delta:
                    # Perform 2-opt swap
                    # Reverse segment from i to j
                    best_path[i:j+1] = best_path[i:j+1][::-1]
                    best_cost -= (current_delta - new_delta)
                    improved = True
                    # First improvement heuristic
                    # break

        # If using First Improvement, uncomment the break above and this break
        # if improved: break

    return best_path, best_cost, evaluated_cycles

def solve(filepath, output_name, time_budget=25):
    print(f"Processing {filepath}...")
    N, dist_matrix = read_graph(filepath)

    global_start_time = time.time()

    best_path = None
    best_cost = float('inf')
    total_evaluations = 0

    # Randomize start nodes to try different initializations
    start_nodes = list(range(1, N + 1))
    random.shuffle(start_nodes)

    iteration = 0

    for start_node in start_nodes:
        # Check global time budget
        if time.time() - global_start_time > time_budget:
            break

        iteration += 1

        # Initial Solution: Nearest Neighbor from random start node
        path = nearest_neighbor(N, dist_matrix, start_node=start_node)

        # Optimization: 2-opt
        # We pass a smaller timeout for the individual run, but the main constraint is the loop
        # Let's give each run enough time to converge (e.g. 5s), but check global time

        # We need to modify two_opt to not take the whole budget if called multiple times
        # Actually, just pass the remaining time
        remaining_time = time_budget - (time.time() - global_start_time)
        if remaining_time <= 0: break

        current_path, current_cost, evals = two_opt(path, dist_matrix, time_limit_seconds=remaining_time)
        total_evaluations += evals

        if current_cost < best_cost:
            best_cost = current_cost
            best_path = current_path
            print(f"  New Best Cost: {best_cost:.2f} (Iter {iteration}, Start Node {start_node})")

        current_elapsed = time.time() - global_start_time
        print(f"    -> Iter {iteration} done. Cost: {current_cost:.2f}. Total Time: {current_elapsed:.2f}s")

    elapsed = time.time() - global_start_time
    print(f"Final Cost: {best_cost:.2f}")
    print(f"Total Evaluated Cycles: {total_evaluations:.2e}")
    print(f"Time Taken: {elapsed:.2f} seconds")
    print(f"Iterations Completed: {iteration}")

    path_str = ", ".join(map(str, best_path))
    with open(output_name, 'w') as f:
        f.write(path_str)

    return best_cost, total_evaluations

if __name__ == "__main__":
    # Hardcoded full paths as requested
    file_A = "/content/drive/My Drive/Colab Notebooks/TSP_1000_euclidianDistance.txt"
    file_B = "/content/drive/My Drive/Colab Notebooks/TSP_1000_randomDistance.txt"

    print(f"Using input files:\n{file_A}\n{file_B}")

    print("--- Solving Graph A ---")
    out_A = "/content/drive/My Drive/Colab Notebooks/solution_A.txt"
    cost_A, evals_A = solve(file_A, out_A, time_budget=29)

    print("\n--- Solving Graph B ---")
    out_B = "/content/drive/My Drive/Colab Notebooks/solution_B.txt"
    cost_B, evals_B = solve(file_B, out_B, time_budget=29)

    # Generate Report Info
    report_path = "/content/drive/My Drive/Colab Notebooks/report_stats.txt"
    with open(report_path, "w") as f:
        f.write(f"Graph A (Euclidean):\n")
        f.write(f"Cost: {cost_A:.2f}\n")
        f.write(f"Evaluated Cycles: {evals_A:.1e}\n\n")
        f.write(f"Graph B (Random):\n")
        f.write(f"Cost: {cost_B:.2f}\n")
        f.write(f"Evaluated Cycles: {evals_B:.1e}\n")

    # Merge solutions into the final file with Student ID
    final_sol_path = "/content/drive/My Drive/Colab Notebooks/solution_925583110.txt"
    with open(final_sol_path, 'w') as outfile:
        with open(out_A, 'r') as infile:
            outfile.write(infile.read() + "\n")
        with open(out_B, 'r') as infile:
            outfile.write(infile.read())

    print(f"\nDone! Solutions saved to {final_sol_path}")

Using input files:
/content/drive/My Drive/Colab Notebooks/TSP_1000_euclidianDistance.txt
/content/drive/My Drive/Colab Notebooks/TSP_1000_randomDistance.txt
--- Solving Graph A ---
Processing /content/drive/My Drive/Colab Notebooks/TSP_1000_euclidianDistance.txt...
  New Best Cost: 2612.03 (Iter 1, Start Node 769)
    -> Iter 1 done. Cost: 2612.03. Total Time: 0.81s
    -> Iter 2 done. Cost: 2619.95. Total Time: 1.73s
  New Best Cost: 2562.89 (Iter 3, Start Node 559)
    -> Iter 3 done. Cost: 2562.89. Total Time: 2.89s
    -> Iter 4 done. Cost: 2571.06. Total Time: 3.74s
    -> Iter 5 done. Cost: 2567.58. Total Time: 4.60s
    -> Iter 6 done. Cost: 2605.52. Total Time: 5.65s
    -> Iter 7 done. Cost: 2601.72. Total Time: 6.58s
    -> Iter 8 done. Cost: 2615.50. Total Time: 7.49s
    -> Iter 9 done. Cost: 2624.36. Total Time: 8.42s
    -> Iter 10 done. Cost: 2586.94. Total Time: 9.66s
    -> Iter 11 done. Cost: 2592.81. Total Time: 10.43s
    -> Iter 12 done. Cost: 2606.31. Total Time: